In [1]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Connection string
server = 'techentdec.database.windows.net'
database = 'QAECECRM_dec'
username = 'dbadmin'
password = 'DashTech1234'
connection_string = f"""
Driver={{ODBC Driver 17 for SQL Server}};
Server={server};
Database={database};
Uid={username};
Pwd={password};
Encrypt=yes;
TrustServerCertificate=no;
Connection Timeout=30;
"""

# Connect to the SQL database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")
    exit()

# Define table names
table_names = [
    '[dbo].[Contract]', '[dbo].[ContractArtist]', '[dbo].[BlueCard]', 
    '[dbo].[LuEventType]', '[dbo].[BlueCardEventDate]'
]

# Initialize an empty dictionary to store DataFrames
dataframes = {}

Connection successful!


In [3]:

# Fetch data from SQL tables
for table in table_names:
    query = f"SELECT * FROM {table}"
    try:
        dataframes[table] = pd.read_sql(query, connection)
        print(f"Successfully imported data from table: {table}")
    except Exception as e:
        print(f"Failed to import data from table: {table}. Error: {e}")

# Assign DataFrames to variables for readability
contract_df = dataframes.get('[dbo].[Contract]', pd.DataFrame())
contract_artist_df = dataframes.get('[dbo].[ContractArtist]', pd.DataFrame())
blue_card_df = dataframes.get('[dbo].[BlueCard]', pd.DataFrame())
event_type_df = dataframes.get('[dbo].[LuEventType]', pd.DataFrame())
blue_card_date_df = dataframes.get('[dbo].[BlueCardEventDate]', pd.DataFrame())

# Data Cleaning
def clean_dataframe(df):
    if df.empty:
        return df
    # Drop duplicates
    df = df.drop_duplicates()
    # Replace empty strings or NaNs with None
    df = df.replace(r'^\s*$', np.nan, regex=True)
    # Standardize column names
    df.columns = df.columns.str.strip().str.lower()
    return df

Successfully imported data from table: [dbo].[Contract]
Successfully imported data from table: [dbo].[ContractArtist]
Successfully imported data from table: [dbo].[BlueCard]
Successfully imported data from table: [dbo].[LuEventType]
Successfully imported data from table: [dbo].[BlueCardEventDate]


In [4]:

# Clean all DataFrames
contract_df = clean_dataframe(contract_df)
contract_artist_df = clean_dataframe(contract_artist_df)
blue_card_df = clean_dataframe(blue_card_df)
event_type_df = clean_dataframe(event_type_df)
blue_card_date_df = clean_dataframe(blue_card_date_df)

# Join tables to create a master dataset
# Merge BlueCard with EventType
bluecard_eventtype = pd.merge(blue_card_df, event_type_df, how='left', left_on='eventtypeid', right_on='eventtypeid')

# Merge Contract with BlueCard
contract_bluecard = pd.merge(contract_df, bluecard_eventtype, how='left', left_on='bluecardid', right_on='bluecardid')

# Merge ContractArtist
full_data = pd.merge(contract_bluecard, contract_artist_df, how='left', left_on='contractid', right_on='contractid')

# Merge BlueCardEventDate
full_data = pd.merge(full_data, blue_card_date_df, how='left', left_on='bluecardid', right_on='bluecardid')

# Handle missing values
full_data.fillna({'gross': 0}, inplace=True)  # Replace numeric NaNs with 0
full_data.fillna('Unknown', inplace=True)  # Replace categorical NaNs with 'Unknown'

# Feature engineering: Add new columns if necessary
full_data['event_year'] = pd.to_datetime(full_data['createddate'], errors='coerce').dt.year


# Save cleaned data to a CSV for future use
full_data.to_csv('cleaned_data.csv', index=False)

# Preview cleaned data
print(full_data.head())


/tmp/ipykernel_249447/3996531290.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  full_data['event_year'] = pd.to_datetime(full_data['createddate'], errors='coerce').dt.year


   contractid  agentid_x officeid  contractstatusid lineofbusinessid  gross_x  \
0      100304          0     17.0                 3              2.0   1700.0   
1      101731        419      1.0                 6              3.0    550.0   
2      103324          0      1.0                 4              3.0   2000.0   
3      103331        313      1.0                 5              3.0   1500.0   
4      103526          0      1.0                 3              3.0   1000.0   

  bluecardid leadsourceid contracttypeid isreseller  ... aref programstotal  \
0    Unknown          3.0            1.0       True  ...  0.0       Unknown   
1    Unknown          5.0            1.0       True  ...  0.0       Unknown   
2    Unknown          5.0            1.0      False  ...  0.0       Unknown   
3    Unknown          5.0            1.0       True  ...  0.0       Unknown   
4    Unknown          5.0            1.0      False  ...  0.0       Unknown   

  eventtimereasonid                 cr

In [5]:

# Replace placeholders
full_data.replace('Unknown', np.nan, inplace=True)

# Drop rows with missing target variable
full_data.dropna(subset=['gross_x'], inplace=True)

# Identify high and low cardinality categorical columns
categorical_columns = full_data.select_dtypes(include=['object']).columns
high_cardinality_cols = [col for col in categorical_columns if full_data[col].nunique() > 50]
low_cardinality_cols = [col for col in categorical_columns if full_data[col].nunique() <= 50]

# Apply label encoding for high-cardinality columns
label_encoder = LabelEncoder()
for col in high_cardinality_cols:
    full_data[col] = label_encoder.fit_transform(full_data[col].astype(str))

# Apply one-hot encoding for low-cardinality columns
full_data = pd.get_dummies(full_data, columns=low_cardinality_cols, drop_first=True)

# Ensure all remaining columns are numeric
full_data = full_data.apply(pd.to_numeric, errors='coerce')

# Fill remaining missing values with 0
full_data.fillna(0, inplace=True)


In [6]:
# Replace placeholders and drop rows with missing target variable
full_data.replace('Unknown', np.nan, inplace=True)
full_data.dropna(subset=['gross_x'], inplace=True)

# Drop irrelevant or high-cardinality columns
columns_to_drop = ['presenteraccountname', 'venuecontactname', 'createdbyid', 'updatedbyid']  # Add other non-essential columns
full_data.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

# Frequency encoding for categorical columns
def frequency_encode(df, column):
    freq = df[column].value_counts()
    df[column] = df[column].map(freq)
    return df

categorical_columns = full_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    full_data = frequency_encode(full_data, col)

# Fill missing values with 0
full_data.fillna(0, inplace=True)

# Ensure all remaining columns are numeric
full_data = full_data.apply(pd.to_numeric, errors='coerce')

# Prepare features (X) and target (y)
X = full_data.drop(['gross_x'], axis=1, errors='ignore')  # Drop the target variable
y = full_data['gross_x']  # Target variable

In [7]:
# Prepare features (X) and target (y)
X = full_data.drop(['gross_x'], axis=1, errors='ignore')  # Drop the target variable
y = full_data['gross_x']  # Target variable

# Split data for ML
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

In [20]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(kernel='linear'),
    "XGBoost": XGBRegressor(random_state=42)
}

# Evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    results[name] = {"MSE": mse, "R-squared": r2}

# Print Results
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  Mean Squared Error: {metrics['MSE']}")
    print(f"  R-squared: {metrics['R-squared']}\n")
